In [ ]:
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
import pandas as pd

In [ ]:
pd.options.display.max_columns = 99
pd.options.display.max_rows = 999

# Requirenments

Description: Emission dashboard to explore individal country contributions, and identify the most polluting nations over time.

- Graph 1: Shows all countries total emissions by population and GDP. Date is adjustable with a slider.  
- Graph 2: Shows detailed emission values for the selected country from Graph 1 over time. There should be an option to select total or per capita values.  
- Graph 3: Shows the top 10 most pollutant nations over time. Option for total or per capita values.

### Information about the combined dataset from the World Bank

Contains data in the following categories:

- Country stats:
  - Population, total
  - GDP per capita (constant 2015 US$)  
    
    
- Emission data:
  - CO2 emissions (kt)
  - CO2 emissions (metric tons per capita)
  - Methane emissions (kt of CO2 equivalent)
  - Nitrous oxide emissions (thousand metric tons of CO2 equivalent)

### Read and clean data

In [ ]:
# Read in data and drop not needed columns
data_path = 'Data Sources/'
df = pd.read_csv(data_path + 'World Development Indicators.csv')
df = df.drop(columns=['Country Code', 'Series Code'])

# Pivot values to long format and shorten year values
df = df.melt(id_vars = ['Country Name', 'Series Name'], var_name = 'Year', value_name = 'Value').dropna()
df['Year'] = df['Year'].str[0:4]
df['Year'] = pd.to_numeric(df['Year'],errors='coerce')
df['Value'] = pd.to_numeric(df['Value'],errors='coerce')

# Simplify indicator notations
indicator_translation = {
    'Population, total' : 'Population',
    'GDP per capita (constant 2015 US$)' : 'GDP (per capita)',
    'CO2 emissions (kt)' : 'CO2',
    'CO2 emissions (metric tons per capita)' : 'CO2 (per capita)',
    'Methane emissions (kt of CO2 equivalent)' : 'CH4',
    'Nitrous oxide emissions (thousand metric tons of CO2 equivalent)' : 'NOX'
}
df['Series Name'].replace(indicator_translation, inplace=True)

### Calculate per capita values where it is missing

In [ ]:
# Get population data, and calculate emission per population, adjust to metric tons
df_emissions_per_capita = (
    df[df['Series Name'].isin(['CH4','NOX'])].merge(
        df[df['Series Name'] == 'Population'], on = ['Country Name', 'Year'], how = 'inner', suffixes = ['','_y'])
    )
df_emissions_per_capita['Value New'] = df_emissions_per_capita['Value'].div(df_emissions_per_capita['Value_y'])*1000

# Prepare new dataset so it can be concatenated to original set
df_emissions_per_capita = df_emissions_per_capita[['Country Name', 'Series Name', 'Year', 'Value New']]
df_emissions_per_capita.rename({'Value New' : 'Value'}, axis = 1, inplace=True)

# Add per capita notations
indicator_translation = {
    'NOX' : 'NOX (per capita)',
    'CH4' : 'CH4 (per capita)'
}
df_emissions_per_capita['Series Name'].replace(indicator_translation, inplace=True)

# Add new per capita values next to the original sub-set of emissions
emissions = ['CO2', 'CO2 (per capita)', 'NOX', 'CH4']
df_emissions = pd.concat([df[df['Series Name'].isin(emissions)],df_emissions_per_capita])

### Rank countries based on total emissions

In [ ]:
# Pivot table to enable total calculation
df_emissions_ranked = (
    df_emissions.pivot(index = ['Country Name', 'Year'], columns = ['Series Name'], values = 'Value')
)
df_emissions_ranked.reset_index(inplace=True)

In [ ]:
# Calculate totals
df_emissions_ranked['Total Emission'] = df_emissions_ranked['CH4']+df_emissions_ranked['CO2']+df_emissions_ranked['NOX']
df_emissions_ranked['Total Emission (per capita)'] = \
    df_emissions_ranked['CH4 (per capita)']+df_emissions_ranked['CO2 (per capita)']+df_emissions_ranked['NOX (per capita)']

# Rank countries by Total Emission for every year
df_emissions_ranked.loc[:, 'Emission Rank'] = df_emissions_ranked.groupby(['Year'])['Total Emission'].rank(ascending=False)
df_emissions_ranked.loc[:, 'Emission Rank (per capita)'] = df_emissions_ranked.groupby(['Year'])['Total Emission (per capita)'].rank(ascending=False)

### Set up dashboard layout

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server

emission_types = {'Carbone dioxide' : 'CO2', 'Methane': 'CH4', 'Nitrous oxide' : 'NOX'}

app.layout = html.Div([            
                html.Div([
                    dcc.Dropdown(
                            id='crossfilter-meta-type',
                            options=[{'label': i, 'value': i} for i in ['Population', 'GDP (per capita)']],
                            value='Population'
                        ),
                    dcc.Graph(
                        id='crossfilter-population-gdp',
                        clickData={'points': [{'customdata': 'Japan'}]}
                    )
                ], style={'width': '100%', 'display': 'inline-block', 'padding': '0 20'}),
    
                html.Div(dcc.Slider(
                    id='crossfilter-year-slider',
                    min=df['Year'].min(),
                    max=df['Year'].max(),
                    value=df['Year'].max(),
                    marks={str(year): str(year) for year in df['Year'].unique()},
                    step=None
                ), style={'width': '100%', 'padding': '0px 20px 20px 20px'}),
                    
                html.Div([
                    html.Div([
                        dcc.Dropdown(
                            id='crossfilter-emission-main',
                            options=[{'label': i, 'value': i} for i in emission_types],
                            value='Carbone dioxide'
                        ),
                        dcc.RadioItems(
                            id='crossfilter-emission-type',
                            options=[{'label': i, 'value': i} for i in ['Total', 'Per capita']],
                            value='Total',
                            labelStyle={'display': 'inline-block'}
                        )
                    ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})

                ], style={
                    'borderBottom': 'thin lightgrey solid',
                    'backgroundColor': 'rgb(250, 250, 250)',
                    'padding': '10px 5px'
                    }
                ),
                    html.Div([
                    dcc.Graph(id='display-emission'),
                ], style={'display': 'inline-block', 'width': '45%'})
            ]
            )

In [ ]:
@app.callback(
    dash.dependencies.Output('crossfilter-population-gdp', 'figure'),
    dash.dependencies.Input('crossfilter-meta-type', 'value')
)
def update_population_gdp(meta_type):
    #dff = df[df['Year'] == year_value]
    #dff = df[df['Country Name'].isin(['Hungary', 'Croatia', 'Albania'])].sort_values(['Country Name','Year'])
    
    dff = df[df['Series Name'] == meta_type].sort_values(['Country Name','Year'])
    dff.loc[dff['Year'] == 1960, 'Value'] = float('nan')
    
    return {
        
        'data': [dict(
            y=dff['Value'],
            x=dff['Year'],
            text=dff['Country Name'],
            customdata=dff['Country Name'],
            mode='lines',
            line={
                'color' : dff['Country Name']
            }
        )],
        'layout': dict(
            xaxis={
                'title': 'Year',
                'type': 'linear'
            },
            yaxis={
                'title': meta_type,
                'type': 'linear'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

In [ ]:
final_page = app.run_server()

In [ ]:
dff = df[df['Country Name'].isin(['Hungary', 'Croatia'])]
dff[dff['Series Name'] == 'Population'].sort_values(['Country Name', 'Year'])

In [ ]:
dff[dff['Series Name'] == 'GDP (per capita)'].sort_values(['Country Name', 'Year'])

In [ ]:
app.layout = html.Div([            
                html.Div([
                    dcc.Graph(
                        id='crossfilter-population-gdp',
                        clickData={'points': [{'customdata': 'Japan'}]}
                    )
                ], style={'width': '100%', 'display': 'inline-block', 'padding': '0 20'}),
    
                html.Div(dcc.Slider(
                    id='crossfilter-year-slider',
                    min=df['Year'].min(),
                    max=df['Year'].max(),
                    value=df['Year'].max(),
                    marks={str(year): str(year) for year in df['Year'].unique()},
                    step=None
                ), style={'width': '100%', 'padding': '0px 20px 20px 20px'}),
                    
                html.Div([
                    html.Div([
                        dcc.Dropdown(
                            id='crossfilter-emission-main',
                            options=[{'label': i, 'value': i} for i in emission_types],
                            value='Carbone dioxide'
                        ),
                        dcc.RadioItems(
                            id='crossfilter-emission-type',
                            options=[{'label': i, 'value': i} for i in ['Total', 'Per capita']],
                            value='Total',
                            labelStyle={'display': 'inline-block'}
                        )
                    ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})

                ], style={
                    'borderBottom': 'thin lightgrey solid',
                    'backgroundColor': 'rgb(250, 250, 250)',
                    'padding': '10px 5px'
                    }
                ),
                    html.Div([
                    dcc.Graph(id='display-emission'),
                ], style={'display': 'inline-block', 'width': '45%'})
            ]
            )

In [ ]:
@app.callback(
    dash.dependencies.Output('crossfilter-population-gdp', 'figure'),
    dash.dependencies.Input('crossfilter-year-slider', 'value')
)
def update_population_gdp(year_value):
    dff = df[df['Year'] == year_value]
    
    return {
        
        'data': [dict(
            x=dff[dff['Series Name'] == 'GDP (per capita)']['Value'],
            y=dff[dff['Series Name'] == 'Population']['Value'],
            text=dff[dff['Series Name'] == 'Population']['Country Name'],
            customdata=dff[dff['Series Name'] == 'Population']['Country Name'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': 'GDP (per capita)',
                'type': 'linear'
            },
            yaxis={
                'title': 'Population',
                'type': 'linear'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

In [ ]:
def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

In [ ]:
@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'clickData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '{}{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'clickData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

In [ ]:
import plotly.express as px

df2 = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(df2, x="year", y="lifeExp", color="country")
fig.show()

In [ ]:
df